# Comparison of spectrum between GIGATL outputs and Alitka Satellite 

Requisites :
 - git clone https://github.com/auraoupa/gonzag_cloud
 


In [1]:
import sys,os
from os import getenv
import warnings
warnings.filterwarnings("ignore")

In [2]:
GONZAG_DIR = '/home/jovyan/gonzag_cloud' ; 
sys.path.append(GONZAG_DIR)
import gonzag as gz


## Params

For the model

In [3]:
model = 'eNATL60'
region = '2'
season = 'fma'
surf='surface_hourly'
name_mod = model+'-Region'+region+'-'+season
name_ssh_mod = 'sossheig'
name_lsm_mod = 'tmask' 
name_lat_mod = 'lat'
name_lon_mod = 'lon'
name_time_mod = 'time_counter'
l_griddist = False
period_mod = ['2010-02-01','2010-04-30']

For altimetry data

In [4]:
name_sat= 'Altika'
tag_sat_catalog = 'al'
name_ssh_sat='sla_unfiltered'
name_time_sat='time'
period_sat = ['2014-02-01','2014-04-30'] #should cover the same period of a different year if needed

## Data

In [5]:
import xarray as xr
import intake
import gcsfs

gcs = gcsfs.GCSFileSystem(requester_pays=True)

cat = intake.open_catalog('../../SWOT-AdAC/catalog.yaml')
dsmod1 = cat[model](region=region,depth=surf, season=season).to_dask()

In [6]:
dsmod1

<xarray.Dataset>
Dimensions:        (time_counter: 2136, x: 607, y: 735)
Coordinates: (12/15)
    depth          (y, x) float32 dask.array<chunksize=(735, 607), meta=np.ndarray>
    e1f            (y, x) float64 dask.array<chunksize=(735, 607), meta=np.ndarray>
    e1t            (y, x) float64 dask.array<chunksize=(735, 607), meta=np.ndarray>
    e1u            (y, x) float64 dask.array<chunksize=(735, 607), meta=np.ndarray>
    e1v            (y, x) float64 dask.array<chunksize=(735, 607), meta=np.ndarray>
    e2f            (y, x) float64 dask.array<chunksize=(735, 607), meta=np.ndarray>
    ...             ...
    lat            (y, x) float32 dask.array<chunksize=(735, 607), meta=np.ndarray>
    lon            (y, x) float32 dask.array<chunksize=(735, 607), meta=np.ndarray>
    nav_lat        (y, x) float32 dask.array<chunksize=(735, 607), meta=np.ndarray>
    nav_lon        (y, x) float32 dask.array<chunksize=(735, 607), meta=np.ndarray>
    time_centered  (time_counter) datetime64[ns] dask.array<chunksize=(72,), meta=np.ndarray>
  * time_counter   (time_counter) datetime64[ns] 2010-02-01T00:30:00 ... 2010...
Dimensions without coordinates: x, y
Data variables: (12/13)
    fmask          (y, x) int8 dask.array<chunksize=(735, 607), meta=np.ndarray>
    qt_oce         (time_counter, y, x) float32 dask.array<chunksize=(72, 735, 607), meta=np.ndarray>
    somecrty       (time_counter, y, x) float32 dask.array<chunksize=(72, 735, 607), meta=np.ndarray>
    sometauy       (time_counter, y, x) float32 dask.array<chunksize=(72, 735, 607), meta=np.ndarray>
    sosaline       (time_counter, y, x) float32 dask.array<chunksize=(72, 735, 607), meta=np.ndarray>
    sossheig       (time_counter, y, x) float32 dask.array<chunksize=(72, 735, 607), meta=np.ndarray>
    ...             ...
    sowaflup       (time_counter, y, x) float32 dask.array<chunksize=(72, 735, 607), meta=np.ndarray>
    sozocrtx       (time_counter, y, x) float32 dask.array<chunksize=(72, 735, 607), meta=np.ndarray>
    sozotaux       (time_counter, y, x) float32 dask.array<chunksize=(72, 735, 607), meta=np.ndarray>
    tmask          (y, x) int8 dask.array<chunksize=(735, 607), meta=np.ndarray>
    umask          (y, x) int8 dask.array<chunksize=(735, 607), meta=np.ndarray>
    vmask          (y, x) int8 dask.array<chunksize=(735, 607), meta=np.ndarray>

In [7]:
if model == 'GIGATL':#get rid of 29th of february (2010 = leap year)
    import numpy as nmp
    from gonzag.io import GetTimeVector
    from gonzag.utils import scan_idx
    rvt = GetTimeVector( dsmod1, name_time_mod, lquiet=True )
    (rtu1,rtu2)=nmp.array(['2000-02-29T00:00:00','2000-03-01T00:00:00'], dtype='datetime64')
    jt1, jt2 = scan_idx( rvt, rtu1, rtu2 )
    dsmod=dsmod1.drop_isel(time=nmp.arange(jt1,jt2+1,1))
else:
    dsmod=dsmod1

In [8]:
from intake import open_catalog
cat = open_catalog("https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/ocean/altimetry.yaml")
dssat  = cat[tag_sat_catalog].to_dask()

In [9]:
dssat

<xarray.Dataset>
Dimensions:         (time: 34340107)
Coordinates:
    latitude        (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    longitude       (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
  * time            (time) datetime64[ns] 2013-03-14T05:44:50.445185 ... 2015...
Data variables:
    cycle           (time) int16 dask.array<chunksize=(34340107,), meta=np.ndarray>
    dac             (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    lwe             (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    mdt             (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    ocean_tide      (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    sla_filtered    (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    sla_unfiltered  (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    track           (time) int16 dask.array<chunksize=(34340107,), meta=np.ndarray>
Attributes: (12/24)
    Conventions:               CF-1.6
    Metadata_Conventions:      Unidata Dataset Discovery v1.0
    cdm_data_type:             Swath
    comment:                   Sea surface height measured by altimeters refe...
    contact:                   servicedesk.cmems@mercator-ocean.eu
    creator_email:             servicedesk.cmems@mercator-ocean.eu
    ...                        ...
    software_version:          6.2_DUACS_DT2018_baseline
    source:                    Altika measurements
    ssalto_duacs_comment:      The reference mission used for the altimeter i...
    standard_name_vocabulary:  NetCDF Climate and Forecast (CF) Metadata Conv...
    summary:                   SSALTO/DUACS Delayed-Time Level-3 sea surface ...
    title:                     DT Altika Global Ocean Along track SSALTO/DUAC...

## Model and Satellite objects

In [10]:
ModelGrid = gz.ModGrid( dsmod, period_mod, name_lon_mod, name_lat_mod , name_time_mod, dsmod, name_lsm_mod, distorded_grid=False )



 *** what we use to define model land-sea mask:
    => "tmask" in dataset 

 *** Skipping computation of angle distortion of the model grid! ("-D" option not invoked)...

 *** About model gridded (source) domain:
     * shape =  (735, 607)
     * horizontal resolution:  0.016690899  degrees or  1.8545257530733943  km
     * Is this a global domain w.r.t longitude:  False
       ==> this is a regional domain
       ==> working in the [0:360] frame...
     * lon_min, lon_max =  306.35 316.59
     * lat_min, lat_max =  30.49 40.49
     * should we pay attention to possible STRONG local distorsion in the grid:  False
     * number of time records of interest for the interpolation to come:  2112
       ==> time record dates: 2010-02-01 to 2010-04-30, included



In [11]:
SatelliteTrack = gz.SatTrack( dssat, period_sat, name_time_sat, name_ssh_sat, domain_bounds=ModelGrid.domain_bounds, l_0_360=ModelGrid.l360 )

 *** [SatTrack()] Analyzing the time vector in dataset ...

 *** About satellite track (target) domain:
     * number of time records of interest for the interpolation to come:  12052
       ==> time record indices: 14883510 to 19033581, included

       separated in 112 tracks


## Process the tracks in parallel (results are saved in separate netcdf files)

In [12]:
if not os.path.exists('../results/results_'+name_sat+'-'+name_mod):
        os.makedirs('../results/results_'+name_sat+'-'+name_mod)

In [13]:
def process_one_track(track):
    tt = "{:02d}".format(track)
    Solution0 = gz.Model2SatTrack( ModelGrid, name_ssh_mod, SatelliteTrack, name_ssh_sat, track )
    c1     = 'Model SSH interpolated in space (' ; c2=') and time on satellite track'
    vvar   = [ 'latitude', 'longitude', name_ssh_mod+'_np'   , name_ssh_mod+'_bl' , name_ssh_sat          , 'distance'                            ]
    vunits = [ 'deg.N'   , 'deg.E'    ,          'm'         ,     'm'            ,    'm'                ,    'km'                               ]
    vlongN = [ 'Latitude', 'Longitude', c1+'nearest-point'+c2,  c1+'bilinear'+c2  , 'Input satellite data', 'Cumulated distance from first point' ]
    from gonzag.config import rmissval
    iw = gz.io.SaveTimeSeries( Solution0.time, \
                             nmp.array( [Solution0.lat, Solution0.lon, Solution0.ssh_mod_np,
                                         Solution0.ssh_mod, Solution0.ssh_sat, Solution0.distance] ), \
                             vvar, '../results/results_'+name_sat+'-'+name_mod+'/result_'+str(tt)+'.nc', time_units='',\
                             vunits=vunits, vlnm=vlongN, missing_val=rmissval )
    return Solution0


In [14]:
import dask
import sys
import dask.bag as db
import numpy as nmp

In [15]:
from dask.distributed import Client

client = Client()
client

Connection method: Cluster object,Cluster type: LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Status: running,Using processes: True
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 64,Total memory: 235.96 GiB
Comm: tcp://127.0.0.1:45297,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 64
Started: Just now,Total memory: 235.96 GiB
Comm: tcp://127.0.0.1:36823,Total threads: 8
Dashboard: http://127.0.0.1:39083/status,Memory: 29.50 GiB
Nanny: tcp://127.0.0.1:41503,


In [16]:
%%time
b = db.from_sequence(nmp.arange(len(SatelliteTrack.index_tracks)), npartitions=64)
results = b.map(lambda x: process_one_track(x))
final = results.compute()

tornado.application - ERROR - Exception in callback functools.partial(<bound method IOLoop._discard_future_result of <tornado.platform.asyncio.AsyncIOLoop object at 0x7f18dcd419d0>>, <Task finished name='Task-10' coro=<Worker.heartbeat() done, defined at /srv/conda/envs/notebook/lib/python3.8/site-packages/distributed/worker.py:962> exception=OSError('Timed out during handshake while connecting to tcp://127.0.0.1:45297 after 10 s')>)
Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.8/site-packages/distributed/comm/core.py", line 321, in connect
    handshake = await asyncio.wait_for(comm.read(), time_left())
  File "/srv/conda/envs/notebook/lib/python3.8/asyncio/tasks.py", line 501, in wait_for
    raise exceptions.TimeoutError()
asyncio.exceptions.TimeoutError

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.8/site-packages/tornado/ioloop.py", line 741, 


 ERROR: mind the indices when calling GetSatSSH() !


 ERROR: mind the indices when calling GetSatSSH() !



distributed.scheduler - ERROR - Couldn't gather keys {"('remainder-a1ed184bcf563e3ecff0cbe4be00b634', 0, 0)": []} state: ['waiting'] workers: []
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: [], ('remainder-a1ed184bcf563e3ecff0cbe4be00b634', 0, 0)
NoneType: None
distributed.scheduler - ERROR - Couldn't gather keys {"('remainder-a1ed184bcf563e3ecff0cbe4be00b634', 0, 0)": []} state: ['waiting'] workers: []
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: [], ('remainder-a1ed184bcf563e3ecff0cbe4be00b634', 0, 0)
NoneType: None
distributed.scheduler - ERROR - Couldn't gather keys {"('remainder-a1ed184bcf563e3ecff0cbe4be00b634', 0, 0)": []} state: ['waiting'] workers: []
NoneType: None
distributed.client - WARNING - Couldn't gather 1 keys, rescheduling {"('remainder-a1ed184bcf563e3ecff0cbe4be00b634', 0, 0)": ()}
distributed.scheduler - ERROR - Workers don't have promised key: [], ('remainder-a1ed184bcf563e3ecff0cbe4be00b634', 0


 ERROR: mind the indices when calling GetSatSSH() !


 ERROR: mind the indices when calling GetSatSSH() !


 ERROR: mind the indices when calling GetSatSSH() !


 ERROR: mind the indices when calling GetSatSSH() !


 ERROR: mind the indices when calling GetSatSSH() !


 ERROR: mind the indices when calling GetSatSSH() !


 ERROR: mind the indices when calling GetSatSSH() !


 ERROR: mind the indices when calling GetSatSSH() !



tornado.application - ERROR - Exception in callback <bound method Worker.trigger_profile of <Worker: 'tcp://127.0.0.1:37465', 4, Status.running, stored: 12, running: 6/8, ready: 0, comm: 0, waiting: -10>>
Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.8/site-packages/tornado/ioloop.py", line 905, in _run
    return self.callback()
  File "/srv/conda/envs/notebook/lib/python3.8/site-packages/distributed/worker.py", line 3161, in trigger_profile
    frames = {ident: frames[ident] for ident in active_threads}
  File "/srv/conda/envs/notebook/lib/python3.8/site-packages/distributed/worker.py", line 3161, in <dictcomp>
    frames = {ident: frames[ident] for ident in active_threads}
KeyError: 139744176485952


...

distributed.nanny - WARNING - Restarting worker



 ERROR: mind the indices when calling GetSatSSH() !


 ERROR: mind the indices when calling GetSatSSH() !


 ERROR: mind the indices when calling GetSatSSH() !


 ERROR: mind the indices when calling GetSatSSH() !


 ERROR: mind the indices when calling GetSatSSH() !


 ERROR: mind the indices when calling GetSatSSH() !


 ERROR: mind the indices when calling GetSatSSH() !


 ERROR: mind the indices when calling GetSatSSH() !


 ERROR: mind the indices when calling GetSatSSH() !


 ERROR: mind the indices when calling GetSatSSH() !


 ERROR: mind the indices when calling GetSatSSH() !


 ERROR: mind the indices when calling GetSatSSH() !


 ERROR: mind the indices when calling GetSatSSH() !



distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.core - ERROR - None
Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.8/site-packages/distributed/core.py", line 575, in handle_stream
    handler(**merge(extra, msg))
  File "/srv/conda/envs/notebook/lib/python3.8/site-packages/distributed/scheduler.py", line 4821, in client_releases_keys
    self.transitions(recommendations)
  File "/srv/conda/envs/notebook/lib/python3.8/site-packages/distributed/scheduler.py", line 6753, in transitions
    self.send_all(client_msgs, worker_msgs)
  File "/srv/conda/envs/notebook/lib/python3.8/site-packages/distributed/scheduler.py", line 5335, in send_all
    w = stream_comms[worker]
KeyError: None
distributed.core - ERROR - E

KilledWorker: ("('lambda-22c552d172fced72cbe9d1bca693002e', 45)", <WorkerState 'tcp://127.0.0.1:40871', name: 5, memory: 0, processing: 22>)

distributed.core - ERROR - Exception while handling op heartbeat_worker
Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.8/site-packages/distributed/core.py", line 498, in handle_comm
    result = handler(comm, **msg)
  File "/srv/conda/envs/notebook/lib/python3.8/site-packages/distributed/scheduler.py", line 3949, in heartbeat_worker
    ws._executing = {
  File "/srv/conda/envs/notebook/lib/python3.8/site-packages/distributed/scheduler.py", line 3950, in <dictcomp>
    parent._tasks[key]: duration for key, duration in executing.items()
KeyError: "('lambda-22c552d172fced72cbe9d1bca693002e', 13)"
tornado.application - ERROR - Exception in callback functools.partial(<bound method IOLoop._discard_future_result of <tornado.platform.asyncio.AsyncIOLoop object at 0x7fb19decba00>>, <Task finished name='Task-811' coro=<Worker.heartbeat() done, defined at /srv/conda/envs/notebook/lib/python3.8/site-packages/distributed/worker.py:962> exception=KeyError("('lambda-